### Calculate graph vectors $\vec{G}=\left[S_{mean},\sigma_S,Q,P_{mean},C_{mean},T,A,E^{glo},E^{loc}_{mean},EC_{mean}\right]$

In [120]:
def calc_graph_vector( filename, thresholds ) :
    #-------------------------------------------------------------------#   
    #    Input arguments:                                               #
    #        filename   – name of file containing connectivity matrix   #
    #        thresholds – list containing thresholds of interest        #
    #-------------------------------------------------------------------#
    #=== import libraries
    import numpy as np
    import bct

    #=== load matrix 
    A_raw = np.loadtxt(filename)
    N = A_raw.shape[0]   # number of nodes
    M_sat = N*(N-1)/2    # max number of connections 

    #=== parameters
    N_rep_louvain = 10   # number of Louvain algorithm repetitions
    gamma = 10           # Louvain resolution parameter
    N_measures = 10      # number of graph measures

    graph_measures = np.zeros([ len(thresholds), N_measures ])      # create empty output matrix
    for thr in range(len(thresholds)) : 
        #=== thresholding 
        A = bct.threshold_proportional( A_raw, p=thresholds[thr], copy=True );
        A[np.nonzero(A<0)] = 0                                      # ensure only positive weights
        M_act = A[np.nonzero(A>0)].shape[0]                         # actual number of nonzero connections
        #=== calculate measures
        #-- mean connection strenght 
        S = np.sum(A)/M_act
        #-- connection strenght std
        Svar = np.std(A[np.nonzero(A)])
        #-- modularity
        [M,Q] = bct.modularity_louvain_und(A, gamma)
        for i in range(N_rep_louvain) :
            [Mt,Qt] = bct.modularity_louvain_und(A, gamma)
            if Qt > Q :
                Q = Qt
                M = Mt
        #-- participation coefficient
        P = np.mean(bct.participation_coef_sign(A, M))
        #-- clustering 
        C = np.mean(bct.clustering_coef_wu(A))
        #-- transitivity 
        T = bct.transitivity_wu(A)
        #-- assortativity
        Asso = bct.assortativity_wei(A)
        #-- global & local efficiency 
        Eglo = bct.efficiency_wei(A)
        Eloc = np.mean(bct.efficiency_wei(A, local=True))
        #-- mean eigenvector centralit
        Eig = np.mean(bct.eigenvector_centrality_und(A))
        #=== write vector to matrix
        graph_measures[thr] = [ S, Svar, Q, P, C, T, Asso, Eglo, Eloc, Eig ]

    #=== save results to file
    np.savetxt( filename[:-4]+'_GV.txt', graph_measures )

In [121]:
import os
thresholds = [ 0.05, 0.075, 0.1, 0.125, 0.15 ]

for filename in files
    #=== do the job
    calc_graph_vector(filename,thresholds)